# Project 3

Using SVD to compress an image. 

- a.The percentage of the storage memory used for p singular values versus the storage memory used for the original picture, for various values of p. 
- b.The error between the matrix A storing the original image and the reduced matrix A_p obtained when the first p singular values are used.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from ipywidgets import interact
from skimage import data
%matplotlib inline

In [2]:
gray_images = {
    'Einstein': np.asarray(Image.open("data\image\Einstein_tongue.jpg")),
     'camera': data.camera(),
     'coins': data.coins()
}
color_images = {
    'girl_small': np.asarray(Image.open("data\image\girl_small.jpg")),
    'cara': np.asarray(Image.open("data\image\cara.jpg")),
    'man': np.asarray(Image.open("data\image\man.jpeg"))
}

In [3]:
def compress_svd(img, p):
    # Perform SVD using np.linalg.svd
    U, s, V = np.linalg.svd(img, full_matrices=False) 
    reconst_matrix = np.dot(U[:,:p],np.dot(np.diag(s[:p]),V[:p,:]))
    return reconst_matrix, s

def compress_gray_images(img_name,Mode):
    error_list = []
    img_list = []
    p_list = []
    compression_ratio_list = []
    
    image = gray_images[img_name]
    m = image.shape[0]
    n = image.shape[1]
    max_p  =int((m*n)/(m+n+1))
    i = 0
    for group in Mode:
        for p in Mode[group]:
            if p > max_p:
                print('compress value k should be less than',max_p)
                return

            reconstruct_img, s = compress_svd(image, p)
            compression_ratio = ((p*(m+n+1))/(m*n)) *100.0
            error = sum(sum((image-reconstruct_img)**2))
            
            img_list.append(reconstruct_img)
            p_list.append(p)
            compression_ratio_list.append(compression_ratio)
            error_list.append(error)

    f_img = plt.figure(figsize=(15,15))
    for i in range(len(img_list)):
        f_img.add_subplot(len(Mode), len(Mode[group]), i+1)
        plt.title('compression ratio = {:.2f}%\n original image: {:d} bytes\n mode {:d}: {:d} bytes\n error: {:}'.format\
                  (compression_ratio_list[i],(m*n),p_list[i],((p_list[i]*(m+n+1))),error_list[i]))
        plt.imshow(img_list[i])
    plt.tight_layout()
    
    f_error, ax_error = plt.subplots()
    ax_error.set_title('{:} image Error'.format(img_name))
    ax_error.set_xlabel('Number of Single Values used')
    ax_error.set_ylabel('Error between compress and original image')
    ax_error.plot(p_list,error_list)
    
def compress_color_images(img_name,Mode):
    error_list = []
    img_list = []
    p_list = []
    compression_ratio_list = []
    
    image = color_images[img_name]
    m = image.shape[0]
    n = image.shape[1]
    max_p  =int((m*n)/(m+n+1))
    i = 0
   
    # splitting the array into three 2D array (svd only apply on 2D array)
    r = image[:,:,0]  # array for R
    g = image[:,:,1]  # array for G
    b = image[:,:,2]  # array for B
    for group in Mode:
        for p in Mode[group]:
            if p > max_p:
                print('compress value k should be less than',max_p)
                return
            # Calculating the svd components for all three arrays
            rr, sr = compress_svd(r,p)
            rg, sg = compress_svd(g,p)
            rb, sb = compress_svd(b,p)

            # Creating a array of zeroes; shape will be same as of image matrix
            rimg = np.zeros(image.shape)
            # Adding matrix for R, G & B in created array
            rimg[:,:,0] = rr
            rimg[:,:,1] = rg
            rimg[:,:,2] = rb
            
            # It will check if any value will be less than 0 will be converted to its absolute
            # and, if any value is greater than 255 than it will be converted to 255
            # because in image array of unit8 can only have value between 0 & 255
            for ind1, row in enumerate(rimg):
                for ind2, col in enumerate(row):
                    for ind3, value in enumerate(col):
                        if value < 0:
                            rimg[ind1,ind2,ind3] = abs(value)
                        if value > 255:
                            rimg[ind1,ind2,ind3] = 255

            # converting the compress image array to uint8 type for further conversion into image object
            reconstruct_img = rimg.astype(np.uint8)
            compression_ratio = (p*(m+n+1))/(m*n)*100
            error = np.linalg.norm(image-reconstruct_img)
            
            img_list.append(reconstruct_img)
            p_list.append(p)
            compression_ratio_list.append(compression_ratio)
            error_list.append(error)
    
    f_img = plt.figure(figsize=(15,15))
    for i in range(len(img_list)):
        f_img.add_subplot(len(Mode), len(Mode[group]), i+1)
        plt.title('compression ratio = {:.2f}%\n original image: {:d} bytes\n mode {:d}: {:d} bytes\n error: {:}'.format\
                  (compression_ratio_list[i],(m*n),p_list[i],((p_list[i]*(m+n+1))),error_list[i]))
        plt.imshow(img_list[i])
    plt.tight_layout()
    
    f_error, ax_error = plt.subplots()
    ax_error.set_title('{:} image Error'.format(img_name))
    ax_error.set_xlabel('Number of Single Values used')
    ax_error.set_ylabel('Error between compress and original image')
    ax_error.plot(p_list,error_list)
    plt.tight_layout()


In [4]:
# return original image max singular value
def get_max_p(image_name, image_type = 'color'):
    if image_type == 'color':
        image = color_images[image_name]
    elif image_type == 'gray':
        image = gray_images[image_name]
    else:
        return 'please set your image type as color or gray'
    m = image.shape[0]
    n = image.shape[1]
    max_p  =int((m*n)/(m+n+1))
    return max_p
get_max_p('cara','color')

442

In [5]:
p_Mode_einstein = {1:[1, 2, 4, 6],2:[8, 10, 12, 14],3:[16, 18, 20, 25],4:[50, 75, 100, 121]}
p_Mode_camera = {1:[1, 20, 40, 60],2:[80, 100, 120, 140],3:[160, 180, 230, 255]}
p_Mode_coins = {1:[1, 2, 4, 6],2:[8, 10, 12, 14],3:[16, 18, 20, 25],4:[50, 80, 130, 169]}
@interact
def show_images(img_name = list(gray_images.keys())):
    if img_name == 'Einstein':
        p_Mode = p_Mode_einstein
    elif img_name == 'camera':
        p_Mode = p_Mode_camera
    elif img_name == 'coins':
        p_Mode = p_Mode_coins
    compress_gray_images(img_name = img_name, Mode = p_Mode)
    

interactive(children=(Dropdown(description='img_name', options=('Einstein', 'camera', 'coins'), value='Einstei…

In [6]:
p_Mode_girl_small = {1:[1, 2, 4, 6],2:[8, 10, 12, 14],3:[16, 18, 20, 25],4:[50, 60, 70, 74]}
p_Mode_cara = {1:[1, 20, 40, 60],2:[100, 150, 200, 250],3:[300, 350, 400, 442]}
p_Mode_man = {1:[1, 20, 40, 60],2:[80, 100, 120, 140],3:[160, 180, 200, 230]}
@interact
def show_images(img_name = list(color_images.keys())):
    if img_name == 'girl_small':
        p_Mode = p_Mode_girl_small
    elif img_name == 'cara':
        p_Mode = p_Mode_cara
    elif img_name == 'man':
        p_Mode = p_Mode_man
    compress_color_images(img_name = img_name, Mode = p_Mode)

interactive(children=(Dropdown(description='img_name', options=('girl_small', 'cara', 'man'), value='girl_smal…